## SageMaker Model Monitor (data quality) for SageMaker Batch Transform Jobs - Model Training & Baselining Pipeline

# Setup: 
 - Upload scipts to S3 for Model Train & Baseline Pipeline (used in Notebook 1)
 - Create IAM Role for Lambda functions (used in Notebook 2)

In [ ]:
# Uncomment if not using latest version of SageMaker SDK
import sys

! pip install --upgrade pip
!{sys.executable} -m pip install sagemaker==2.114.0
!{sys.executable} -m pip install -U boto3

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

In [ ]:
import os
import time
import boto3
import sagemaker

In [ ]:
# Setup SageMaker variables
from sagemaker.workflow.pipeline_context import PipelineSession
session = PipelineSession()
bucket = session.default_bucket()
role_arn= sagemaker.get_execution_role()
region = session.boto_region_name
sagemaker_client = boto3.client('sagemaker')
aws_account_id = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
# Setup S3 paths for pipeline inputs, outputs, & artifacts
print("Demo Bucket: {}".format(bucket))
bucket_prefix = f"aim321-demo-model-monitor-batch-transform"

In [ ]:
# Upload Data Preprocessing Script to S3 
s3_client = boto3.client("s3", region_name=boto3.Session().region_name)

processing_script = "preprocessing.py"
evaluation_script = "evaluation.py"

s3_client.upload_file(
    Filename=f"pipeline_scripts/{processing_script}",
    Bucket=bucket,
    Key=f"{bucket_prefix}/scripts/{processing_script}",
)
s3_client.upload_file(
    Filename=f"pipeline_scripts/{evaluation_script}",
    Bucket=bucket,
    Key=f"{bucket_prefix}/scripts/{evaluation_script}",
)

# Create IAM Role

Enable "IAMFullAccess", "AWSLambda_FullAccess", "CloudWatchFullAccess" in IAM for SageMaker Execution Role

In [ ]:
# Create Lambda execution role for Lambda Function using helper function
from iam_helper import create_lambda_role

lambda_role = create_lambda_role("Lambda-SageMaker-GetModelRole") 
print('Lambda Role:', lambda_role)

In [ ]:
### Delete pipeline when needed for cleanup 

sagemaker_client.delete_pipeline(PipelineName='aim321-train-baseline-pipeline-1')